In [ ]:
## V3SciXAngle Investigation

#This notebook is for investigating how to calculate `V3SciXAngle` in the siaf_alignment files.
#In Cox & Lallo (2017), `V3SciXAngle` is defined as "Angle from the V3 axis to the Sci x axis (degrees) at the reference point measured from V3 toward V2." It then states "These values are derivable from the Sci2Idl coefficients and V3IdlYang and V3IdlParity."

#Based on above, the first thing I'll try is to derive `V3SciXAngle` by first using `pysiaf.utils.tools.convert_polynomial_coefficients` and then adding or subtracting the `V3IdlYAngle`.

In [10]:
import numpy as np
import pysiaf

In [4]:
niriss = pysiaf.Siaf('NIRISS')
nis_cen = niriss['NIS_CEN']

In [5]:
coeffs = nis_cen.get_polynomial_coefficients()
A = coeffs['Sci2IdlX']
B = coeffs['Sci2IdlY']
C = coeffs['Idl2SciX']
D = coeffs['Idl2SciY']

In [6]:
poly_coeffs = pysiaf.utils.tools.convert_polynomial_coefficients(A, B, C, D)
poly_coeffs

(array([-0.00000000e+00,  6.53922932e-02, -0.00000000e+00, -2.30483923e-08,
        -5.37698669e-09, -2.51296580e-09, -2.30078479e-12,  6.31369124e-12,
        -8.08542171e-12,  8.48226257e-13,  1.66637652e-15,  8.18777858e-15,
         1.36996132e-15,  7.30563470e-15, -2.57386549e-16]),
 array([ 0.00000000e+00,  2.98555119e-04,  6.58885546e-02, -4.95344067e-08,
        -2.72416385e-08, -4.19312656e-09,  1.37580870e-12, -1.39988279e-11,
         1.80873498e-12, -2.80413971e-11,  1.58408148e-15,  1.08885266e-15,
         8.98048431e-15, -2.00010677e-16,  1.64817452e-14]),
 array([-0.00000000e+00,  1.52922284e+01,  7.56481028e-05,  8.15983179e-05,
         1.87402374e-05,  9.31216768e-06,  1.29274616e-07, -3.49953296e-07,
         4.38997784e-07, -3.76992158e-08, -1.34860405e-09, -6.70880207e-09,
        -1.05985661e-09, -5.80198093e-09,  6.24489048e-11]),
 array([ 0.00000000e+00, -6.93932290e-02,  1.51773082e+01,  1.75103122e-04,
         9.64268032e-05,  1.51910166e-05, -6.75926333e-08

In [11]:
#The `poly_coeffs` above is a tuple of arrays and floats with:
#```AR, BR, CR, DR, V3SciXAngle, V3SciYAngle, V2Ref, V3Ref```

#Note that `(V2Ref, V3Ref) = (0, 0)` above, which means the distortion coefficients were derived by forcing the SIAF alignments to be zero (the alignements parameters are separated into a difference file). The `V3IdlYAngle` is also forced to be zero (very close to zero in above case at least). By definition, `V3SciYAngle = V3IdlYAngle`, so the question is whether we just apply the same rotation to the `V3SciXAngle` above to calculate the final `V3SciXAngle`.

In [8]:
# Based on the "niriss_siaf_alignment.txt" file, V3IdlYAngle = 0.5698701825418142.
# Let's simply try adding that value to the V3SciXAngle above.
poly_coeffs[4] + 0.5698701825418142

-89.16854196662794

In [9]:
# The "niriss_siaf_alignment.txt" file lists V3SciXAngle = -89.69171766828843
# So, simply adding V3IdlYAngle does definitely NOT give the V3SciXAngle value.
# In fact, the original poly_coeffs[4] is CLOSER to the V3SciXAngle listed in "niriss_siaf_alignment.txt" file.
# Let's take the difference between the two:
poly_coeffs[4] - (-89.69171766828843)

-0.046694480881328104

In [ ]:
# Where does the difference above come from?